## Lens API

In [1]:
import json
import requests
search_url = 'https://api.lens.org/patent/search'

In [2]:
## Get API authorisation code
api_auth = open("api_auth.json", "r")
lens_auth = json.load(api_auth)['lens']

api_auth.close()

In [14]:
def get_jurisdictions(countries):
    query_start = '''
                {
                    "bool": {
                        "should": [
            '''

    query_end = '''
                        ]
                    }
                },
            '''

    ## iterate through list of countries and builk query for each country it
    query_builder = []
    for c in countries:
        juridiction =   '''
                        {
                            "term" : {
                                "jurisdiction": "%s"
                            }
                        }
                        ''' % (c)
        
        query_builder.append(juridiction)

    ##
    jurisdictions = (",".join(query_builder))
    query = query_start + jurisdictions + query_end


    return query

In [18]:
query = get_jurisdictions(['US','AU'])
print(query)


                {
                    "bool": {
                        "should": [
            
                        {
                            "term" : {
                                "jurisdiction": "US"
                            }
                        }
                        ,
                        {
                            "term" : {
                                "jurisdiction": "AU"
                            }
                        }
                        
                        ]
                    }
                },
            


In [23]:
def term_builder(term, term_list):
    query_start = '''
                {
                    "bool": {
                        "should": [
            '''

    query_end = '''
                        ]
                    }
                },
            '''

    ## iterate through list of countries and builk query for each country it
    query_builder = []
    for i in term_list:
        term_string =   '''
                            {
                                "term" : {
                                    "%s": "%s"
                                }
                            }
                        ''' % (term, i)
        
        query_builder.append(term_string)

    ## join country list into one string and append to query for return
    terms = (",".join(query_builder))
    query = query_start + terms + query_end

    return query

In [24]:
query = term_builder('juridictions', ['US','AU'])
print(query)


                {
                    "bool": {
                        "should": [
            
                            {
                                "term" : {
                                    "juridictions": "US"
                                }
                            }
                        ,
                            {
                                "term" : {
                                    "juridictions": "AU"
                                }
                            }
                        
                        ]
                    }
                },
            


In [3]:
start_date = "2023-01-01"
end_date = "2023-12-31"
start_from = 0
size = 100      ### For paid licences change this number to 1,000 - 10,000

data = '''{
    "query": {
        "bool": {
            "must": [
                {
                    "bool": {
                        "should": [
                            {
                                "term" : {
                                    "publication_type": "GRANTED_PATENT"
                                }
                            },
                            {
                                "term" : {
                                    "publication_type": "GRANTED_PATENT"
                                }
                            }
                        ]
                    }
                },
                {
                    "bool": {
                        "should": [
                            {
                                "term" : {
                                  "jurisdiction": "US"
                                }
                            },
                            {
                                "term" : {
                                  "jurisdiction": "AU"
                                }
                            }
                        ]
                    }
                },
                {
                    "range" : {
                        "date_published": {
                            "gte": "%s",
                            "lte": "%s"
                        }
                    }
                
                }
            ]
        }
    },
    "sort": [{"date_published": "asc"}],
    "include": ["lens_id",  
                "abstract.text", 
                "date_published", 
                "publication_type",
                "biblio.application_reference.doc_number",
                "biblio.invention_title.text", 
                "biblio.classifications_cpc.classifications.symbol", 
                "biblio.parties.inventors.residence",
                "biblio.parties.applicants.residence"
                ],
    "from" : %d,
    "size" : %d
}''' % (start_date, end_date, start_from, size)

### Request query terms: https://docs.api.lens.org/request-patent.html
### Response column structure: https://docs.api.lens.org/response-patent.html

headers = {'Authorization': lens_auth, 'Content-Type': 'application/json'}
response = requests.post(search_url, data=data, headers=headers)
if response.status_code != requests.codes.ok:
  print(response.status_code)
else:
  print(response.text)

{"total":205334,"max_score":0.0,"data":[{"lens_id":"005-261-557-878-900","date_published":"2023-01-03","biblio":{"invention_title":[{"text":"Link bracelet"}],"parties":{"applicants":[{"residence":"CH"}],"inventors":[{"residence":"FR"}]},"classifications_cpc":{"classifications":[{"symbol":"A44C5/107"},{"symbol":"A44C5/107"},{"symbol":"A44C5/107"},{"symbol":"A44C5/18"}]}},"abstract":[{"text":"A link bracelet includes two rows of side links forming the edges of the bracelet and at least one row of central links arranged between the side links, wherein the side links each include at least two through holes in the inner face thereof facing the central links and the central links are traversed by at least two housings. The bracelet includes arbors inserted into each of the holes in the side links, the head of the arbors being fitted into the housings of the central links and each having a grooved portion in proximity to the end thereof, and the central links include an opening giving access 

In [20]:
response.json()

{'total': 205334,
 'max_score': 0.0,
 'data': [{'lens_id': '005-261-557-878-900',
   'date_published': '2023-01-03',
   'biblio': {'invention_title': [{'text': 'Link bracelet'}],
    'parties': {'applicants': [{'residence': 'CH'}],
     'inventors': [{'residence': 'FR'}]},
    'classifications_cpc': {'classifications': [{'symbol': 'A44C5/107'},
      {'symbol': 'A44C5/107'},
      {'symbol': 'A44C5/107'},
      {'symbol': 'A44C5/18'}]}},
   'abstract': [{'text': 'A link bracelet includes two rows of side links forming the edges of the bracelet and at least one row of central links arranged between the side links, wherein the side links each include at least two through holes in the inner face thereof facing the central links and the central links are traversed by at least two housings. The bracelet includes arbors inserted into each of the holes in the side links, the head of the arbors being fitted into the housings of the central links and each having a grooved portion in proximity t

In [24]:
f = open("../data/raw/" + f"{start_date}_to_{end_date}.json", "w", encoding='utf-8')
f.write(response.text)
f.close()